In [4]:
# Riot LoL Minimal Flow — Get Match History via Riot ID + Match-V5
# Uses ONLY:
#  - /riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}
#  - /lol/match/v5/matches/by-puuid/{puuid}/ids
# -------------------------------------------------------------

from datetime import datetime, timezone
import requests
import time
import pandas as pd
import os

# =============================
# CONFIG
# =============================
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <-- replace with your Riot API key
GAME_NAME = "Hide on bush"       # Riot ID name
TAG_LINE = "KR1"          # Riot ID tagline
REGION = "ASIA"           # choose from: AMERICAS, EUROPE, ASIA
QUEUE = 420               # optional queue filter (420 = ranked solo); set to None for all
OUT_DIR = "output_minimal"
os.makedirs(OUT_DIR, exist_ok=True)

SLEEP_BETWEEN_CALLS = 1.2
RETRY_MAX = 5

# =============================
# HELPERS
# =============================
def _headers():
    return {"X-Riot-Token": API_KEY}

def _request_json(url, params=None):
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=30)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_CALLS)
                return r.json()
            elif r.status_code in (429, 503):
                delay = float(r.headers.get("Retry-After", 1.5 * (2 ** attempt)))
                print(f"Rate limited, sleeping {delay:.1f}s...")
                time.sleep(delay)
            elif r.status_code in (401, 403):
                raise RuntimeError(f"{r.status_code} {r.text[:200]} → Check API key validity or region access.")
            elif r.status_code == 404:
                return None
            else:
                print(f"HTTP {r.status_code}: {r.text[:120]}")
                time.sleep(2)
        except requests.RequestException as e:
            print("Network error:", e)
            time.sleep(2)
    raise RuntimeError(f"Request failed after {RETRY_MAX} attempts.")

# =============================
# STEP 1 — Resolve Riot ID → PUUID (Account-V1)
# =============================
def get_puuid_from_riotid(gameName, tagLine, region):
    url = f"https://{region.lower()}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"
    data = _request_json(url)
    if not data or "puuid" not in data:
        raise RuntimeError(f"Failed to resolve Riot ID {gameName}#{tagLine}. Response: {data}")
    puuid = data["puuid"]
    print(f"✅ {gameName}#{tagLine} → PUUID: {puuid}")
    return puuid

# =============================
# STEP 2 — Get Match IDs (Match-V5)
# =============================
def get_all_match_ids(puuid, region, queue=None):
    base_url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    start = 0
    all_ids = []

    while True:
        params = {"start": start, "count": 100}
        if queue is not None:
            params["queue"] = queue
        ids = _request_json(base_url, params=params)
        if not ids:
            break
        all_ids.extend(ids)
        print(f"Fetched {len(ids)} IDs (total {len(all_ids)})")
        start += len(ids)

    print(f"✅ Total matches found: {len(all_ids)}")
    return all_ids

# =============================
# RUN
# =============================
puuid = get_puuid_from_riotid(GAME_NAME, TAG_LINE, REGION)
match_ids = get_all_match_ids(puuid, REGION, queue=None)

# Save to CSV
out_path = os.path.join(OUT_DIR, f"match_ids_{GAME_NAME}_{TAG_LINE}.csv")
pd.DataFrame({"matchId": match_ids}).to_csv(out_path, index=False)
print("Saved match IDs to:", out_path)

# preview
pd.read_csv(out_path).head()


✅ Hide on bush#KR1 → PUUID: nrlGgzDDgaxt5Xw8kHKTKBExb81_VfVWrtwOiKJ3l47WJngAxULpg4SwueYHIvK1606gzmHhnyt60A
Fetched 100 IDs (total 100)
Fetched 100 IDs (total 200)
Fetched 85 IDs (total 285)


KeyboardInterrupt: 

In [15]:
# =============================
# STEP 3 — Fetch Details for First 10 Matches
# =============================
import json, os

def get_match_detail(match_id, region):
    """Fetch full match JSON for a given matchId from Match-V5."""
    url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    data = _request_json(url)
    if not data:
        print(f"⚠️ Skipping {match_id} (no data)")
        return None
    return data

# Limit to first 10 matches for testing
sample_match_ids = match_ids[:10]
print(f"Fetching details for {len(sample_match_ids)} matches...")

details = []
for i, mid in enumerate(sample_match_ids, start=1):
    info = get_match_detail(mid, REGION)
    if info:
        details.append(info)
    print(f"  [{i}/{len(sample_match_ids)}] Done {mid}")
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Pulled {len(details)} match details")

# Save raw JSON (optional)
out_json = os.path.join(OUT_DIR, f"match_details_sample_{GAME_NAME}_{TAG_LINE}.json")
with open(out_json, "w") as f:
    json.dump(details, f)
print("Saved sample match data to:", out_json)


Fetching details for 10 matches...
  [1/10] Done KR_7768456553
  [2/10] Done KR_7768428775
  [3/10] Done KR_7737155171
  [4/10] Done KR_7694218699
  [5/10] Done KR_7669256754
  [6/10] Done KR_7669237789
  [7/10] Done KR_7669222323
  [8/10] Done KR_7669206621
  [9/10] Done KR_7636932338
  [10/10] Done KR_7599078493
✅ Pulled 10 match details
Saved sample match data to: output_minimal/match_details_sample_Faker_KR1.json


In [16]:
# =============================
# STEP 4 — Fetch Match Timelines (Events / Gold / XP)
# =============================
import json, os

def get_match_timeline(match_id, region):
    """Fetch timeline JSON for a given matchId."""
    url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"
    data = _request_json(url)
    if not data:
        print(f"⚠️ Skipping timeline for {match_id}")
        return None
    return data

# Limit to first 3–5 matches for testing (timeline is large)
timeline_match_ids = match_ids[:5]
timelines = []

for i, mid in enumerate(timeline_match_ids, start=1):
    timeline = get_match_timeline(mid, REGION)
    if timeline:
        timelines.append(timeline)
    print(f"  [{i}/{len(timeline_match_ids)}] timeline done for {mid}")
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Pulled {len(timelines)} match timelines")

# Save raw JSON
out_timeline_json = os.path.join(OUT_DIR, f"match_timelines_sample_{GAME_NAME}_{TAG_LINE}.json")
with open(out_timeline_json, "w") as f:
    json.dump(timelines, f)
print("Saved timeline data to:", out_timeline_json)


  [1/5] timeline done for KR_7768456553
  [2/5] timeline done for KR_7768428775
  [3/5] timeline done for KR_7737155171
  [4/5] timeline done for KR_7694218699
  [5/5] timeline done for KR_7669256754
✅ Pulled 5 match timelines
Saved timeline data to: output_minimal/match_timelines_sample_Faker_KR1.json


In [18]:
timeline_rows = []
for tl in timelines:
    match_id = tl["metadata"]["matchId"]
    for frame in tl["info"]["frames"]:
        tstamp = frame["timestamp"] // 60000  # minutes
        for pid, pdata in frame["participantFrames"].items():
            timeline_rows.append({
                "matchId": match_id,
                "minute": tstamp,
                "participantId": pid,
                "totalGold": pdata["totalGold"],
                "xp": pdata["xp"],
                "level": pdata["level"],
            })
timeline_df = pd.DataFrame(timeline_rows)
timeline_df


,matchId,minute,participantId,totalGold,xp,level
0,KR_7768456553,0,1,500,0,1
1,KR_7768456553,0,2,500,0,1
2,KR_7768456553,0,3,500,0,1
3,KR_7768456553,0,4,500,0,1
4,KR_7768456553,0,5,500,0,1
...,...,...,...,...,...,...
1005,KR_7669256754,23,6,17067,28684,18
1006,KR_7669256754,23,7,14813,26515,18
1007,KR_7669256754,23,8,16864,27554,18
1008,KR_7669256754,23,9,18826,30208,18


In [20]:
player_df = timeline_df[timeline_df['participantId'] == "1"]
player_df

,matchId,minute,participantId,totalGold,xp,level
0,KR_7768456553,0,1,500,0,1
10,KR_7768456553,1,1,905,18,1
20,KR_7768456553,2,1,1073,267,1
30,KR_7768456553,3,1,1993,915,3
40,KR_7768456553,4,1,3015,1556,4
...,...,...,...,...,...,...
960,KR_7669256754,20,1,14716,25973,18
970,KR_7669256754,21,1,15121,26615,18
980,KR_7669256754,22,1,15796,28492,18
990,KR_7669256754,23,1,16156,28792,18


In [10]:
# Flatten participant data into a table
participants = []
for match in details:
    info = match["info"]
    for p in info["participants"]:
        participants.append({
            "matchId": info["gameId"],
            "gameDuration": info["gameDuration"],
            "queueId": info["queueId"],
            "championName": p["championName"],
            "kills": p["kills"],
            "deaths": p["deaths"],
            "assists": p["assists"],
            "win": p["win"],
            "summonerName": p["summonerName"],
            "teamId": p["teamId"],
            "role": p["role"],
            "lane": p["lane"]
        })

df = pd.DataFrame(participants)
print("Participants shape:", df.shape)
df


Participants shape: (106, 12)


,matchId,gameDuration,queueId,championName,kills,deaths,assists,win,summonerName,teamId,role,lane
0,5403251060,1690,420,Rumble,4,6,8,True,,100,SOLO,TOP
1,5403251060,1690,420,Ambessa,13,5,9,True,,100,NONE,JUNGLE
2,5403251060,1690,420,Ahri,4,6,11,True,,100,SOLO,MIDDLE
3,5403251060,1690,420,Riven,14,3,10,True,,100,SOLO,BOTTOM
4,5403251060,1690,420,Blitzcrank,2,3,25,True,,100,NONE,JUNGLE
...,...,...,...,...,...,...,...,...,...,...,...,...
101,5402460752,1096,420,Gwen,5,1,3,True,,200,SUPPORT,NONE
102,5402460752,1096,420,Khazix,10,0,1,True,,200,SUPPORT,NONE
103,5402460752,1096,420,Ahri,4,0,7,True,,200,SUPPORT,NONE
104,5402460752,1096,420,Smolder,2,3,3,True,,200,SUPPORT,NONE


In [11]:
df_match = df[df['matchId'] == 5403251060]
df_match

,matchId,gameDuration,queueId,championName,kills,deaths,assists,win,summonerName,teamId,role,lane
0,5403251060,1690,420,Rumble,4,6,8,True,,100,SOLO,TOP
1,5403251060,1690,420,Ambessa,13,5,9,True,,100,NONE,JUNGLE
2,5403251060,1690,420,Ahri,4,6,11,True,,100,SOLO,MIDDLE
3,5403251060,1690,420,Riven,14,3,10,True,,100,SOLO,BOTTOM
4,5403251060,1690,420,Blitzcrank,2,3,25,True,,100,NONE,JUNGLE
5,5403251060,1690,420,Nidalee,4,6,10,False,,200,SOLO,TOP
6,5403251060,1690,420,JarvanIV,11,7,5,False,,200,NONE,JUNGLE
7,5403251060,1690,420,Mel,7,7,2,False,,200,SOLO,MIDDLE
8,5403251060,1690,420,Jhin,0,8,11,False,,200,CARRY,BOTTOM
9,5403251060,1690,420,Soraka,1,9,5,False,,200,SUPPORT,BOTTOM


# champion-v3

In [2]:
# =============================
# STEP 5 — Champion Rotations (Champion-V3)
# =============================
def get_champion_rotations(platform):
    """
    Fetch current free champion rotation list.
    platform = e.g. 'na1', 'euw1', 'kr', 'br1'
    """
    url = f"https://{platform}.api.riotgames.com/lol/platform/v3/champion-rotations"
    data = _request_json(url)
    if not data:
        print(f"⚠️ No data returned for champion rotations on {platform}")
        return None

    free = data.get("freeChampionIds", [])
    new = data.get("freeChampionIdsForNewPlayers", [])
    print(f"✅ {len(free)} free champions this week on {platform}")
    print(f"🆕 {len(new)} for new players")
    return data

# Example: check NA1 rotation
rotations = get_champion_rotations("na1")
rotations


✅ 20 free champions this week on na1
🆕 20 for new players


{'freeChampionIds': [5,
  23,
  25,
  29,
  31,
  53,
  55,
  84,
  85,
  106,
  119,
  133,
  154,
  157,
  163,
  360,
  421,
  432,
  516,
  777],
 'freeChampionIdsForNewPlayers': [17,
  18,
  33,
  37,
  51,
  54,
  80,
  82,
  89,
  91,
  112,
  113,
  115,
  131,
  134,
  145,
  222,
  254,
  350,
  875],
 'maxNewPlayerLevel': 10}

In [3]:
def get_champion_list():
    url = "https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json"
    return requests.get(url).json()["data"]

champions = get_champion_list()
id_to_name = {int(v["key"]): k for k, v in champions.items()}

free_names = [id_to_name.get(cid, cid) for cid in rotations["freeChampionIds"]]
print("Free champions this week:", free_names)


Free champions this week: ['XinZhao', 'Tryndamere', 'Morgana', 'Twitch', 'Chogath', 'Blitzcrank', 'Katarina', 'Akali', 'Kennen', 'Volibear', 'Draven', 'Quinn', 'Zac', 'Yasuo', 'Taliyah', 'Samira', 'RekSai', 'Bard', 'Ornn', 'Yone']


# Summoner V4

In [3]:
import requests, time

API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <-- rotate & paste new key
PUUID   = "0KeRPKJZA9Y6ufAqpft8_N-VpTu6lc1vWrJRpN03_nqXb8znm2Q8vDtpPrltWv8EsEviF4h4LsSOIA"                          # the puuid you have

AMERICAS_PLATFORMS = ["na1", "br1", "la1", "la2", "oc1"]
EUROPE_PLATFORMS   = ["euw1", "eun1", "tr1", "ru"]
ASIA_PLATFORMS     = ["kr", "jp1"]
ALL_PLATFORMS = AMERICAS_PLATFORMS + EUROPE_PLATFORMS + ASIA_PLATFORMS

def _get(url, params=None, tries=3):
    for i in range(tries):
        r = requests.get(url, headers={"X-Riot-Token": API_KEY}, params=params, timeout=20)
        if r.status_code == 200:
            return r.json()
        if r.status_code == 429:
            time.sleep(float(r.headers.get("Retry-After", 1.5*(2**i))))
            continue
        if r.status_code in (401,403):
            raise RuntimeError(f"{r.status_code}: {r.text[:200]} (auth/permissions)")
        if r.status_code == 404:
            return None
        # other codes: brief backoff and retry
        time.sleep(1.5*(2**i))
    raise RuntimeError(f"Failed after {tries} tries")

def summoner_by_puuid(platform, puuid):
    url = f"https://{platform}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
    return _get(url)

# Probe all platforms to find where this PUUID has a Summoner object
found = None
for plat in ALL_PLATFORMS:
    data = summoner_by_puuid(plat, PUUID)
    if data:
        found = (plat, data)
        print(f"✅ Found on {plat}: name={data.get('name')} level={data.get('summonerLevel')}")
        break
    else:
        print(f"… not on {plat}")

if not found:
    raise SystemError("Could not find a Summoner for this PUUID on any platform. "
                      "Double-check the PUUID (is it LoL, not another Riot game?), "
                      "or resolve PUUID via Account-V1 using the player’s Riot ID, then retry.")

# Use the found platform info:
platform, summoner = found
print("Summoner object keys:", list(summoner.keys()))
print("summonerId:", summoner.get("id"))
print("accountId:", summoner.get("accountId"))
print("name:", summoner.get("name"))
print("level:", summoner.get("summonerLevel"))


… not on na1
… not on br1
… not on la1
… not on la2
… not on oc1
… not on euw1
… not on eun1
… not on tr1
… not on ru
✅ Found on kr: name=None level=22
Summoner object keys: ['puuid', 'profileIconId', 'revisionDate', 'summonerLevel']
summonerId: None
accountId: None
name: None
level: 22


# league-v4

In [5]:
# === Cell 0: Setup (auto-detect platforms + queues) ===
import os, time, requests, pandas as pd
from typing import Optional, Dict, Any, List

# --- CONFIG ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"   # your Riot key
PUUID   = None                                           # optional (for later cells)
RETRY_MAX = 5
SLEEP_BETWEEN_REQUESTS = 1.0

# --- Possible platform hosts ---
AMERICAS_PLATFORMS = ["na1","br1","la1","la2","oc1"]
EUROPE_PLATFORMS   = ["euw1","eun1","tr1","ru"]
ASIA_PLATFORMS     = ["kr","jp1"]
ALL_PLATFORMS      = AMERICAS_PLATFORMS + EUROPE_PLATFORMS + ASIA_PLATFORMS

# --- Ranked queues to test ---
ALL_QUEUES = ["RANKED_SOLO_5x5", "RANKED_FLEX_SR", "RANKED_TFT"]

# --- Helpers ---
def _headers() -> Dict[str,str]:
    return {"X-Riot-Token": API_KEY}

def riot_get(url: str, params: Optional[Dict[str,Any]] = None) -> Any:
    last_err = None
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=10)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_REQUESTS)
                return r.json()
            if r.status_code == 429:
                delay = float(r.headers.get("Retry-After", 1.5*(2**attempt)))
                print(f"429 rate-limited; sleeping {delay:.1f}s")
                time.sleep(delay); continue
            if r.status_code in (401,403):
                raise RuntimeError(f"{r.status_code}: auth/permissions error → {r.text[:200]}")
            if r.status_code == 404:
                return None
            last_err = f"{r.status_code}: {r.text[:100]}"
        except requests.RequestException as e:
            last_err = str(e)
        time.sleep(1.0*(2**attempt))
    raise RuntimeError(f"Request failed after retries. Last error: {last_err}")

def entries_to_df(entries: List[dict]) -> pd.DataFrame:
    if not entries: return pd.DataFrame()
    df = pd.DataFrame(entries)
    cols = [c for c in [
        "summonerName","summonerId","leaguePoints","wins","losses",
        "rank","tier","queueType","veteran","inactive","hotStreak","freshBlood","miniSeries"
    ] if c in df.columns]
    return df[cols] if cols else df

# --- Step 1: Platform connectivity check ---
reachable = []
print("🔍 Checking Riot platform hosts...\n")
for plat in ALL_PLATFORMS:
    url = f"https://{plat}.api.riotgames.com/lol/status/v4/platform-data"
    try:
        r = requests.get(url, headers=_headers(), timeout=6)
        if r.status_code == 200:
            reachable.append(plat)
            print(f"✅ {plat} → OK")
        elif r.status_code == 403:
            print(f"❌ {plat} → 403 Forbidden (key expired or not authorized)")
            break
        else:
            print(f"⚠️ {plat} → {r.status_code}")
    except Exception as e:
        print(f"⚠️ {plat} → {e}")

if not reachable:
    raise SystemExit("No reachable platforms — check your API key or network.")
PLATFORM = reachable[0]
print(f"\n➡️ Using first reachable platform: {PLATFORM}")

# --- Step 2: Queue validation on that platform ---
valid_queues = []
print("\n🔍 Checking available queues on", PLATFORM, "...")
for q in ALL_QUEUES:
    url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{q}"
    r = requests.get(url, headers=_headers(), timeout=8)
    if r.status_code == 200:
        valid_queues.append(q)
        print(f"✅ {q} available")
    elif r.status_code == 404:
        print(f"⚠️ {q} not supported on this platform")
    else:
        print(f"⚠️ {q} returned {r.status_code}")

if not valid_queues:
    raise SystemExit("No queues returned successfully — check key or platform access.")
QUEUE = valid_queues[0]
print(f"\n➡️ Using first available queue: {QUEUE}")
print("\nConfig ready ✅")


🔍 Checking Riot platform hosts...

✅ na1 → OK
✅ br1 → OK
✅ la1 → OK
✅ la2 → OK
✅ oc1 → OK
✅ euw1 → OK
✅ eun1 → OK
✅ tr1 → OK
✅ ru → OK
✅ kr → OK
✅ jp1 → OK

➡️ Using first reachable platform: na1

🔍 Checking available queues on na1 ...
✅ RANKED_SOLO_5x5 available
✅ RANKED_FLEX_SR available
⚠️ RANKED_TFT returned 400

➡️ Using first available queue: RANKED_SOLO_5x5

Config ready ✅


In [6]:
# === Cell 1: /lol/league/v4/challengerleagues/by-queue/{queue} ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{QUEUE}"
challenger = riot_get(url)
if not challenger:
    raise SystemExit("No challenger data returned.")

print(f"TIER: {challenger.get('tier')}  |  QUEUE: {challenger.get('queue')}")
chal_df = entries_to_df(challenger.get("entries", []))
print("Challenger entries:", chal_df.shape)
chal_df.head(10)


TIER: CHALLENGER  |  QUEUE: RANKED_SOLO_5x5
Challenger entries: (300, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,2185,509,367,I,True,False,False,False
1,1895,552,458,I,True,False,False,False
2,1782,414,335,I,True,False,False,False
3,1630,347,278,I,True,False,False,False
4,1629,280,178,I,True,False,False,False
5,1597,690,613,I,True,False,False,False
6,1594,313,214,I,True,False,False,False
7,1560,140,76,I,False,False,False,False
8,1522,354,283,I,True,False,False,False
9,1510,287,220,I,True,False,False,False


In [7]:
# === Cell 2: /lol/league/v4/grandmasterleagues/by-queue/{queue} ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{QUEUE}"
grandmaster = riot_get(url)
if not grandmaster:
    raise SystemExit("No grandmaster data returned.")

print(f"TIER: {grandmaster.get('tier')}  |  QUEUE: {grandmaster.get('queue')}")
gm_df = entries_to_df(grandmaster.get("entries", []))
print("Grandmaster entries:", gm_df.shape)
gm_df.head(10)


TIER: GRANDMASTER  |  QUEUE: RANKED_SOLO_5x5
Grandmaster entries: (700, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,911,992,963,I,False,False,True,True
1,852,744,704,I,True,False,True,False
2,852,154,98,I,False,False,True,True
3,850,876,843,I,False,False,False,True
4,850,407,368,I,True,False,False,False
5,848,695,653,I,False,False,False,True
6,838,595,555,I,False,False,False,True
7,835,393,362,I,False,False,False,True
8,832,232,157,I,False,False,False,True
9,829,370,332,I,False,False,False,True


In [8]:
# === Cell 3: /lol/league/v4/masterleagues/by-queue/{queue} ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{QUEUE}"
master = riot_get(url)
if not master:
    raise SystemExit("No master data returned.")

print(f"TIER: {master.get('tier')}  |  QUEUE: {master.get('queue')}")
master_df = entries_to_df(master.get("entries", []))
print("Master entries:", master_df.shape)
master_df.head(10)


TIER: MASTER  |  QUEUE: RANKED_SOLO_5x5
Master entries: (6290, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,548,141,89,I,True,False,True,False
1,522,393,374,I,False,False,True,True
2,519,85,60,I,True,False,False,False
3,518,456,443,I,False,False,True,True
4,515,82,13,I,False,False,True,False
5,508,638,645,I,True,False,True,False
6,505,495,463,I,False,False,False,True
7,505,220,182,I,False,False,True,True
8,504,383,355,I,False,False,True,True
9,503,434,397,I,False,False,True,True


In [9]:
# === Cell 4: /lol/league/v4/entries/{queue}/{tier}/{division}?page=N ===
TIER = "DIAMOND"   # change
DIV  = "I"         # I, II, III, IV

all_pages = []
page = 1
while True:
    url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/entries/{QUEUE}/{TIER}/{DIV}"
    entries = riot_get(url, params={"page": page})
    if not entries:
        break
    all_pages.extend(entries)
    print(f"Loaded page {page}: {len(entries)} rows")
    page += 1
    if page > 5:  # safety cap; remove/raise if you want all pages
        break

entries_df = entries_to_df(all_pages)
print(f"Total {TIER} {DIV} entries:", entries_df.shape)
entries_df.head(10)


Loaded page 1: 205 rows
Loaded page 2: 205 rows
Loaded page 3: 205 rows
Loaded page 4: 205 rows
Loaded page 5: 205 rows
Total DIAMOND I entries: (1025, 10)


,leaguePoints,wins,losses,rank,tier,queueType,veteran,inactive,hotStreak,freshBlood
0,57,263,269,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,True
1,75,40,30,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
2,25,39,54,I,DIAMOND,RANKED_SOLO_5x5,False,False,True,False
3,84,109,93,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
4,75,143,137,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,True
5,56,253,267,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,True
6,97,109,83,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
7,8,61,57,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
8,5,190,184,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
9,56,249,248,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False


In [10]:
# === Cell 5: /lol/league/v4/leagues/{leagueId} ===
league_id = (challenger or {}).get("leagueId", None)
if not league_id:
    raise SystemExit("No leagueId found (run Cell 1 first).")

url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/leagues/{league_id}"
league_detail = riot_get(url)
print("League:", league_detail.get("tier"), "|", league_detail.get("queue"))
ld_df = entries_to_df(league_detail.get("entries", []))
print("League detail entries:", ld_df.shape)
ld_df.head(10)


League: CHALLENGER | RANKED_SOLO_5x5
League detail entries: (300, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,2185,509,367,I,True,False,False,False
1,1895,552,458,I,True,False,False,False
2,1782,414,335,I,True,False,False,False
3,1630,347,278,I,True,False,False,False
4,1629,280,178,I,True,False,False,False
5,1597,690,613,I,True,False,False,False
6,1594,313,214,I,True,False,False,False
7,1560,140,76,I,False,False,False,False
8,1522,354,283,I,True,False,False,False
9,1510,287,220,I,True,False,False,False


In [11]:
# === Cell 6: /lol/league/v4/entries/by-puuid/{encryptedPUUID} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 to use this call.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/entries/by-puuid/{PUUID}"
by_puuid = riot_get(url) or []
print(f"Ranked entries for this player: {len(by_puuid)}")

# Normalize list of dicts → DataFrame
puuid_df = pd.DataFrame(by_puuid)
# reorder common columns if present
cols = [c for c in [
    "queueType","tier","rank","leaguePoints","wins","losses","hotStreak","veteran","inactive","freshBlood"
] if c in puuid_df.columns]
puuid_df = puuid_df[cols] if cols else puuid_df
puuid_df.head(10)


SystemExit: Set PUUID in Cell 0 to use this call.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# === Cell 7: Save JSON & CSV snapshots (optional) ===
import json, os
os.makedirs("output_league_v4", exist_ok=True)

snap = {
    "challenger": challenger,
    "grandmaster": grandmaster,
    "master": master,
    "entries_diamond_I": entries_df.to_dict(orient="records"),
    "league_detail": league_detail if "league_detail" in locals() else None,
    "by_puuid": by_puuid if "by_puuid" in locals() else None,
}
with open("output_league_v4/all_league_v4.json", "w") as f:
    json.dump(snap, f, indent=2)

# Also save a few CSVs
chal_df.to_csv("output_league_v4/challenger_entries.csv", index=False)
gm_df.to_csv("output_league_v4/grandmaster_entries.csv", index=False)
master_df.to_csv("output_league_v4/master_entries.csv", index=False)
entries_df.to_csv("output_league_v4/diamond_I_entries.csv", index=False)
if "puuid_df" in locals():
    puuid_df.to_csv("output_league_v4/by_puuid_entries.csv", index=False)

print("Saved outputs in output_league_v4/")


# League v4 exp

In [13]:
# === League-EXP-V4: entries across ALL platforms & queues (no manual division config) ===
import requests, time, pandas as pd, os

API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <-- put your key here

# Platforms to scan
AMERICAS = ["na1","br1","la1","la2","oc1"]
EUROPE   = ["euw1","eun1","tr1","ru"]
ASIA     = ["kr","jp1"]
ALL_PLATFORMS = AMERICAS + EUROPE + ASIA

# LoL ranked queues (TFT uses a different product; leave it out here)
ALL_QUEUES = ["RANKED_SOLO_5x5", "RANKED_FLEX_SR"]

# Tiers
DIV_TIERS  = ["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
APEX_TIERS = ["MASTER","GRANDMASTER","CHALLENGER"]  # division ignored by Riot for these

# Request controls
RETRY_MAX = 5
BASE_SLEEP = 1.0     # base pacing between calls
OUT_DIR = "output_league_exp_v4_all"

def _get(url, params=None):
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers={"X-Riot-Token": API_KEY}, params=params, timeout=25)
        if r.status_code == 200:
            time.sleep(BASE_SLEEP)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…")
            time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions issue → {r.text[:180]}")
        if r.status_code == 404:  # unsupported combo on that platform/queue
            return None
        last = f"{r.status_code}: {r.text[:180]}"
        time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

def fetch_entries(platform:str, queue:str, tier:str, division:str|None, max_pages:int=50):
    """
    Fetches all pages for a given (platform, queue, tier[, division]).
    For apex tiers, 'division' can be None (we send 'I' only to satisfy the path).
    """
    div_for_path = division if division is not None else "I"   # Riot ignores division for apex, but path requires it
    base = f"https://{platform}.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{div_for_path}"

    rows = []
    page = 1
    while page <= max_pages:
        data = _get(base, params={"page": page})
        if not data:
            break
        # annotate each row
        for d in data:
            d["_platform"] = platform
            d["_queue"]    = queue
            d["_tier"]     = tier
            d["_division"] = division if division is not None else "NA"  # NA = not applicable (apex)
        rows.extend(data)
        # If a page returns < 205 entries, it's usually the last; still try next once
        print(f"{platform} | {queue} | {tier} {div_for_path}: page {page} → {len(data)} rows (total {len(rows)})")
        if len(data) == 0:
            break
        page += 1
        # gentle pacing between pages
        time.sleep(0.3)
    return rows

# -------- Run the sweep ----------
all_rows = []

for plat in ALL_PLATFORMS:
    # quick connectivity probe (optional but helpful)
    probe = requests.get(f"https://{plat}.api.riotgames.com/lol/status/v4/platform-data",
                         headers={"X-Riot-Token": API_KEY}, timeout=8)
    if probe.status_code != 200:
        print(f"Skip {plat} (status={probe.status_code})")
        continue

    for q in ALL_QUEUES:
        # First check if the queue exists on this platform by requesting Challenger apex page 1
        chk = requests.get(
            f"https://{plat}.api.riotgames.com/lol/league-exp/v4/entries/{q}/CHALLENGER/I",
            headers={"X-Riot-Token": API_KEY}, params={"page": 1}, timeout=12
        )
        if chk.status_code not in (200, 404):  # non-fatal but informative
            print(f"  {plat} {q} probe → {chk.status_code}")
        if chk.status_code == 404:
            print(f"  {plat} does not support {q}; skipping.")
            continue

        # 1) Divisional tiers (I..IV)
        for tier in DIV_TIERS:
            for div in ["I","II","III","IV"]:
                try:
                    rows = fetch_entries(plat, q, tier, div, max_pages=50)
                    all_rows.extend(rows)
                except Exception as e:
                    print(f"Error on {plat} {q} {tier} {div}: {e}")

        # 2) Apex tiers (division not applicable; we label 'NA')
        for tier in APEX_TIERS:
            try:
                rows = fetch_entries(plat, q, tier, division=None, max_pages=50)
                all_rows.extend(rows)
            except Exception as e:
                print(f"Error on {plat} {q} {tier}: {e}")

print(f"\nTotal rows collected: {len(all_rows)}")

# -------- DataFrame + save ----------
df = pd.DataFrame(all_rows)
if not df.empty:
    # Clean and order some useful columns if present
    useful = [c for c in [
        "_platform","_queue","_tier","_division",
        "summonerName","summonerId","queueType","tier","rank",
        "leaguePoints","wins","losses","hotStreak","veteran","inactive","freshBlood"
    ] if c in df.columns]
    df = df[useful + [c for c in df.columns if c not in useful]]  # keep extras too

print("DataFrame shape:", df.shape)
display(df.head(20))

os.makedirs(OUT_DIR, exist_ok=True)
csv_path = os.path.join(OUT_DIR, "league_exp_all_platforms_queues.csv")
df.to_csv(csv_path, index=False)
print("Saved →", csv_path)


na1 | RANKED_SOLO_5x5 | IRON I: page 1 → 205 rows (total 205)
na1 | RANKED_SOLO_5x5 | IRON I: page 2 → 205 rows (total 410)
na1 | RANKED_SOLO_5x5 | IRON I: page 3 → 205 rows (total 615)
na1 | RANKED_SOLO_5x5 | IRON I: page 4 → 205 rows (total 820)
na1 | RANKED_SOLO_5x5 | IRON I: page 5 → 205 rows (total 1025)
na1 | RANKED_SOLO_5x5 | IRON I: page 6 → 205 rows (total 1230)
na1 | RANKED_SOLO_5x5 | IRON I: page 7 → 205 rows (total 1435)
na1 | RANKED_SOLO_5x5 | IRON I: page 8 → 205 rows (total 1640)
na1 | RANKED_SOLO_5x5 | IRON I: page 9 → 205 rows (total 1845)
na1 | RANKED_SOLO_5x5 | IRON I: page 10 → 205 rows (total 2050)
na1 | RANKED_SOLO_5x5 | IRON I: page 11 → 205 rows (total 2255)
na1 | RANKED_SOLO_5x5 | IRON I: page 12 → 205 rows (total 2460)
na1 | RANKED_SOLO_5x5 | IRON I: page 13 → 205 rows (total 2665)
na1 | RANKED_SOLO_5x5 | IRON I: page 14 → 205 rows (total 2870)
na1 | RANKED_SOLO_5x5 | IRON I: page 15 → 205 rows (total 3075)
na1 | RANKED_SOLO_5x5 | IRON I: page 16 → 205 rows (t

KeyboardInterrupt: 

this is one of Riot’s data-heavy ranked-ladder APIs. It exposes a paginated listing of every player in a given ranked queue, tier, and division — effectively the “expanded” version of the standard League-V4 /entries call.

# Clash v1

In [14]:
# === Cell 0: Setup (config + platform check + helpers) ===
import os, time, requests, pandas as pd
from typing import Optional, Dict, Any, List

# --- CONFIG (edit these) ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"   # paste your key
PUUID   = None   # set when using Cell 1 (players/by-puuid)
TEAM_ID = None   # set when using Cells 2 & 5
TOURNAMENT_ID = None  # set when using Cell 4

RETRY_MAX = 5
SLEEP_BETWEEN_REQUESTS = 1.0

# Platform list (Clash is platform-routed)
AMERICAS_PLATFORMS = ["na1","br1","la1","la2","oc1"]
EUROPE_PLATFORMS   = ["euw1","eun1","tr1","ru"]
ASIA_PLATFORMS     = ["kr","jp1"]
ALL_PLATFORMS      = AMERICAS_PLATFORMS + EUROPE_PLATFORMS + ASIA_PLATFORMS

def _headers() -> Dict[str, str]:
    return {"X-Riot-Token": API_KEY}

def riot_get(url: str, params: Optional[Dict[str, Any]] = None) -> Any:
    last_err = None
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=20)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_REQUESTS)
                return r.json()
            if r.status_code == 429:
                delay = float(r.headers.get("Retry-After", 1.5 * (2 ** attempt)))
                print(f"429 rate-limited; sleeping {delay:.1f}s")
                time.sleep(delay); continue
            if r.status_code in (401, 403):
                raise RuntimeError(f"{r.status_code}: auth/permissions → {r.text[:200]}")
            if r.status_code == 404:
                return None
            last_err = f"{r.status_code}: {r.text[:160]}"
        except requests.RequestException as e:
            last_err = str(e)
        time.sleep(1.0 * (2 ** attempt))
    raise RuntimeError(f"Request failed after retries. Last error: {last_err}")

# Auto-pick the first reachable platform (status endpoint)
reachable = []
for plat in ALL_PLATFORMS:
    try:
        r = requests.get(f"https://{plat}.api.riotgames.com/lol/status/v4/platform-data",
                         headers=_headers(), timeout=8)
        if r.status_code == 200:
            reachable.append(plat)
            print(f"✅ {plat} OK")
        elif r.status_code == 403:
            print(f"❌ {plat} 403 (key expired/forbidden)"); break
        else:
            print(f"⚠️  {plat} → {r.status_code}")
    except Exception as e:
        print(f"⚠️  {plat} error:", e)

if not reachable:
    raise SystemExit("No reachable platforms — check API key/network.")
PLATFORM = reachable[0]
print(f"\nUsing PLATFORM → {PLATFORM}")


✅ na1 OK
✅ br1 OK
✅ la1 OK
✅ la2 OK
✅ oc1 OK
✅ euw1 OK
✅ eun1 OK
✅ tr1 OK
✅ ru OK
✅ kr OK
✅ jp1 OK

Using PLATFORM → na1


In [15]:
# === Cell 1: /lol/clash/v1/players/by-puuid/{puuid} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 to query players/by-puuid.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/players/by-puuid/{PUUID}"
player_entries = riot_get(url) or []   # list of PlayerDTOs

df_players = pd.DataFrame(player_entries)
print("players/by-puuid rows:", df_players.shape)
df_players.head(10)


SystemExit: Set PUUID in Cell 0 to query players/by-puuid.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
# === Cell 2: /lol/clash/v1/teams/{teamId} ===
if not TEAM_ID:
    raise SystemExit("Set TEAM_ID in Cell 0 to query teams/{teamId}.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/teams/{TEAM_ID}"
team = riot_get(url)
if not team:
    raise SystemExit("No team data returned (404 or empty).")

# Basic fields
team_df = pd.json_normalize({
    "teamId": team.get("id"),
    "tournamentId": team.get("tournamentId"),
    "name": team.get("name"),
    "iconId": team.get("iconId"),
    "tier": team.get("tier"),
    "captain": team.get("captain"),
    "abbreviation": team.get("abbreviation")
})
print("Team info:")
display(team_df)

# Roster (players array)
players_list = team.get("players", []) or []
players_df = pd.json_normalize(players_list)
print("Team players:")
players_df.head(10)


SystemExit: Set TEAM_ID in Cell 0 to query teams/{teamId}.

In [17]:
# === Cell 3: /lol/clash/v1/tournaments ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/tournaments"
tournaments = riot_get(url) or []  # list of TournamentDTO

# Base tournament info
base_cols = ["id","themeId","name"]
base_rows = [{k: t.get(k) for k in base_cols} for t in tournaments]
tournaments_df = pd.DataFrame(base_rows)
print("Tournaments:", tournaments_df.shape)
display(tournaments_df.head(10))

# Explode schedules into a table (one row per schedule entry)
sched_rows = []
for t in tournaments:
    for s in (t.get("schedule") or []):
        row = {"tournamentId": t.get("id")}
        row.update(s)  # usually has "id","registrationTime","startTime","cancelled"
        sched_rows.append(row)
schedules_df = pd.DataFrame(sched_rows)
print("Tournament schedules:", schedules_df.shape)
schedules_df.head(10)


Tournaments: (2, 3)


,id,themeId,name
0,136381,9,None
1,136382,9,None


Tournament schedules: (2, 5)


,tournamentId,id,registrationTime,startTime,cancelled
0,136381,129241,1763856900000,1763866800000,False
1,136382,129242,1763943300000,1763953200000,False


In [18]:
# === Cell 4: /lol/clash/v1/tournaments/{tournamentId} ===
if not TOURNAMENT_ID:
    raise SystemExit("Set TOURNAMENT_ID in Cell 0 to query tournaments/{tournamentId}.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/tournaments/{TOURNAMENT_ID}"
t = riot_get(url)
if not t:
    raise SystemExit("No tournament data returned (404 or empty).")

# Base fields
tournament_df = pd.json_normalize({k: t.get(k) for k in ["id","themeId","name"]})
print("Tournament:")
display(tournament_df)

# Schedule (explode)
sched_df = pd.DataFrame(t.get("schedule") or [])
print("Schedule entries:")
sched_df.head(10)


SystemExit: Set TOURNAMENT_ID in Cell 0 to query tournaments/{tournamentId}.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
# === Cell 5: /lol/clash/v1/tournaments/by-team/{teamId} ===
if not TEAM_ID:
    raise SystemExit("Set TEAM_ID in Cell 0 to query tournaments/by-team/{teamId}.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/tournaments/by-team/{TEAM_ID}"
list_for_team = riot_get(url) or []  # list of TournamentDTOs

team_tourneys_df = pd.DataFrame([{k: t.get(k) for k in ["id","themeId","name"]} for t in list_for_team])
print("Tournaments for team:", team_tourneys_df.shape)
display(team_tourneys_df.head(10))

# All schedules combined
sched_rows = []
for t in list_for_team:
    for s in (t.get("schedule") or []):
        row = {"tournamentId": t.get("id"), "name": t.get("name")}
        row.update(s)
        sched_rows.append(row)
team_sched_df = pd.DataFrame(sched_rows)
print("Schedules for team:", team_sched_df.shape)
team_sched_df.head(10)


SystemExit: Set TEAM_ID in Cell 0 to query tournaments/by-team/{teamId}.